Calculates discounted cashflows for Fixed-Floating and OIS (TRAINING DATA)

In [94]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os

import pandas as pd
import os

#r'C:\Users\gusta\Documents\KTH\TriOptima\trioptima/'
#'/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/'
your_path = r'C:\Users\gusta\Documents\KTH\TriOptima\trioptima/'
folder_path = your_path + '6.Active Data/Train Model Data/'
# Get file in the folder
files = os.listdir(folder_path)
# MAC issue
files = [f for f in files if f != '.DS_Store']
data_file = files[0][:-4]

# Load the data
data = pd.read_csv(your_path + "2.Cleaned/" + data_file +'_Cleaned.csv')

# Filter trades based on conditions
filtered_data = data[
    (data['leg1NotionalCurrency'].isin(['EUR', 'USD', 'GBP'])) & 
    (data['leg1UnderlyingAssetOrContractType'].isin(['Fixed-Floating', 'OIS']))
]

print(filtered_data['leg2UnderlierID'].unique())

df = pd.DataFrame(filtered_data)

# Load the exchange rates
exchange_rates_df = pd.read_csv(your_path + 'exchange_rates.csv')
exchange_rates = dict(zip(exchange_rates_df['Currency'], exchange_rates_df['Rate_to_USD']))

def convert_to_usd(row):
    return row['leg1NotionalAmount'] * exchange_rates.get(row['leg1NotionalCurrency'], 1)

df['leg1NotionalAmountUSD'] = df.apply(convert_to_usd, axis=1)

# Read in the external CSV with floating rates
euribor_df = pd.read_csv(your_path + 'EURIBOR.csv')

# Convert date columns to datetime type
df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
df['expirationDate'] = pd.to_datetime(df['expirationDate'])

def calculate_cashflows(row, period, freq):
    cashflows = []
    start_date = row['effectiveDate']
    end_date = row['expirationDate']
    
    if row[period] == 'MNTH':
        delta = relativedelta(months=row[freq])
    elif row[period] == 'YEAR':
        delta = relativedelta(years=row[freq])
    elif row[period] == 'DAIL':
        delta = timedelta(days=row[freq])
    else:
        raise ValueError(f"Unknown frequency period: {row[period]}")
    
    while start_date < end_date:
        cashflows.append(start_date)
        start_date += delta

    return cashflows

df['cashflow_dates'] = df.apply(
    lambda row: 
        calculate_cashflows(row, 'leg1FixedRatePaymentFrequencyPeriod', 'leg1FixedRatePaymentFrequencyMultiplier') 
        if row['leg1UnderlyingAssetOrContractType'] == 'Fixed-Floating' 
        else (calculate_cashflows(row, 'leg2UnderlierTenorPeriod', 'leg2UnderlierTenorMultiplier') 
              if row['leg1UnderlyingAssetOrContractType'] == 'OIS' 
              else None), 
    axis=1)

def convert_to_months(value):
    if "month" in value:
        return int(value.split()[0])
    elif "year" in value:
        return int(value.split()[0]) * 12
    else:
        return 0

euribor_df['MonthsToMaturity'] = euribor_df['timeToMaturity'].apply(convert_to_months)
float_rates_df = euribor_df[['MonthsToMaturity', 'Rate']].copy()

def discount_rate_curve(date):
    months_to_maturity = (date - datetime.now()).days // 30
    if months_to_maturity < 0:
        months_to_maturity = float_rates_df['MonthsToMaturity'].min()
    reindexed_df = float_rates_df.set_index('MonthsToMaturity').reindex(float_rates_df['MonthsToMaturity'], method='nearest')
    rate = reindexed_df['Rate'].get(months_to_maturity, reindexed_df['Rate'].iloc[0])
    return rate / 100

def get_floating_rate(months_to_maturity):
    nearest_index = float_rates_df['MonthsToMaturity'].sub(months_to_maturity).abs().idxmin()
    return float_rates_df.loc[nearest_index, 'Rate']

def calculate_discounted_cashflow(row, type):
    total_discounted_cashflow = 0
    months_to_maturity = (row['expirationDate'] - row['effectiveDate']).days // 30
    for date in row['cashflow_dates']:
        if type == 'float':
            float_rate = get_floating_rate(months_to_maturity) / 100
            cashflow = row['leg1NotionalAmount'] * float_rate
        else:
            cashflow = row['leg1NotionalAmount'] * row['leg1FixedRate']
        time_difference = (date - datetime.now()).days / 365.0
        discounted_cashflow = cashflow / (1 + discount_rate_curve(date))**time_difference
        total_discounted_cashflow += discounted_cashflow
    return total_discounted_cashflow

df['MtM_leg1'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'fixed'), axis=1)
df['MtM_leg2'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'float'), axis=1)


# 1. Define a bumped discount rate curve function:
def bumped_discount_rate_curve(date):
    months_to_maturity = (date - datetime.now()).days // 30
    if months_to_maturity < 0:
        months_to_maturity = float_rates_df['MonthsToMaturity'].min()
    reindexed_df = float_rates_df.set_index('MonthsToMaturity').reindex(float_rates_df['MonthsToMaturity'], method='nearest')
    # Bump the rate by 0.0001
    bumped_rate = reindexed_df['Rate'].get(months_to_maturity, reindexed_df['Rate'].iloc[0]) + 0.01
    return bumped_rate / 100

# 2. Recalculate leg2 value using the bumped rate:
def calculate_bumped_discounted_cashflow(row, type):
    total_discounted_cashflow = 0
    months_to_maturity = (row['expirationDate'] - row['effectiveDate']).days // 30
    for date in row['cashflow_dates']:
        if type == 'float':
            float_rate = get_floating_rate(months_to_maturity) / 100
            cashflow = row['leg1NotionalAmount'] * float_rate
        else:
            cashflow = row['leg1NotionalAmount'] * row['leg1FixedRate']
        time_difference = (date - datetime.now()).days / 365.0
        discounted_cashflow = cashflow / (1 + bumped_discount_rate_curve(date))**time_difference
        total_discounted_cashflow += discounted_cashflow
    return total_discounted_cashflow

df['MtM_leg2_bumped'] = df.apply(lambda row: calculate_bumped_discounted_cashflow(row, 'float'), axis=1)

# 3. Calculate the risk:
df['total_delta'] = (df['MtM_leg2'] - df['MtM_leg2_bumped']).abs()


df.drop(columns=['cashflow_dates'], inplace=True)
df.drop(columns=['MtM_leg2_bumped'], inplace=True)


df.to_csv(your_path + '3.Cash_Risk/' + data_file + '_Cash_Risk.csv', index=False)





['USD-LIBOR-BBA' 'EUR-EURIBOR' 'USD-Federal Funds-H.15-OIS-COMPOUND'
 'EUR-EuroSTR-OIS Compound' 'GBP-SONIA-OIS Compound'
 'USD-SOFR-OIS Compound' 'USD-SOFR-1M']


C:\Users\gusta\AppData\Local\Temp\ipykernel_16960\1285611815.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
C:\Users\gusta\AppData\Local\Temp\ipykernel_16960\1285611815.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['expirationDate'] = pd.to_datetime(df['expirationDate'])


Now the same for the Test data

In [95]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os

import pandas as pd
import os

folder_path = your_path + '6.Active Data/Test Data/'
# Get file in the folder
files = os.listdir(folder_path)
# MAC issue
files = [f for f in files if f != '.DS_Store']
data_file = files[0][:-4]

# Load the data
data = pd.read_csv(your_path + "2.Cleaned/" + data_file +'_Cleaned.csv')

# Filter trades based on conditions
filtered_data = data[
    (data['leg1NotionalCurrency'].isin(['EUR', 'USD', 'GBP'])) & 
    (data['leg1UnderlyingAssetOrContractType'].isin(['Fixed-Floating', 'OIS']))
]

print(filtered_data['leg2UnderlierID'].unique())

df = pd.DataFrame(filtered_data)

# Load the exchange rates
exchange_rates_df = pd.read_csv(your_path + 'exchange_rates.csv')
exchange_rates = dict(zip(exchange_rates_df['Currency'], exchange_rates_df['Rate_to_USD']))

def convert_to_usd(row):
    return row['leg1NotionalAmount'] * exchange_rates.get(row['leg1NotionalCurrency'], 1)

df['leg1NotionalAmountUSD'] = df.apply(convert_to_usd, axis=1)

# Read in the external CSV with floating rates
euribor_df = pd.read_csv(your_path + 'EURIBOR.csv')

# Convert date columns to datetime type
df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
df['expirationDate'] = pd.to_datetime(df['expirationDate'])

def calculate_cashflows(row, period, freq):
    cashflows = []
    start_date = row['effectiveDate']
    end_date = row['expirationDate']
    
    if row[period] == 'MNTH':
        delta = relativedelta(months=row[freq])
    elif row[period] == 'YEAR':
        delta = relativedelta(years=row[freq])
    elif row[period] == 'DAIL':
        delta = timedelta(days=row[freq])
    else:
        raise ValueError(f"Unknown frequency period: {row[period]}")
    
    while start_date < end_date:
        cashflows.append(start_date)
        start_date += delta

    return cashflows

df['cashflow_dates'] = df.apply(
    lambda row: 
        calculate_cashflows(row, 'leg1FixedRatePaymentFrequencyPeriod', 'leg1FixedRatePaymentFrequencyMultiplier') 
        if row['leg1UnderlyingAssetOrContractType'] == 'Fixed-Floating' 
        else (calculate_cashflows(row, 'leg2UnderlierTenorPeriod', 'leg2UnderlierTenorMultiplier') 
              if row['leg1UnderlyingAssetOrContractType'] == 'OIS' 
              else None), 
    axis=1)

def convert_to_months(value):
    if "month" in value:
        return int(value.split()[0])
    elif "year" in value:
        return int(value.split()[0]) * 12
    else:
        return 0

euribor_df['MonthsToMaturity'] = euribor_df['timeToMaturity'].apply(convert_to_months)
float_rates_df = euribor_df[['MonthsToMaturity', 'Rate']].copy()

def discount_rate_curve(date):
    months_to_maturity = (date - datetime.now()).days // 30
    if months_to_maturity < 0:
        months_to_maturity = float_rates_df['MonthsToMaturity'].min()
    reindexed_df = float_rates_df.set_index('MonthsToMaturity').reindex(float_rates_df['MonthsToMaturity'], method='nearest')
    rate = reindexed_df['Rate'].get(months_to_maturity, reindexed_df['Rate'].iloc[0])
    return rate / 100

def get_floating_rate(months_to_maturity):
    nearest_index = float_rates_df['MonthsToMaturity'].sub(months_to_maturity).abs().idxmin()
    return float_rates_df.loc[nearest_index, 'Rate']

def calculate_discounted_cashflow(row, type):
    total_discounted_cashflow = 0
    months_to_maturity = (row['expirationDate'] - row['effectiveDate']).days // 30
    for date in row['cashflow_dates']:
        if type == 'float':
            float_rate = get_floating_rate(months_to_maturity) / 100
            cashflow = row['leg1NotionalAmount'] * float_rate
        else:
            cashflow = row['leg1NotionalAmount'] * row['leg1FixedRate']
        time_difference = (date - datetime.now()).days / 365.0
        discounted_cashflow = cashflow / (1 + discount_rate_curve(date))**time_difference
        total_discounted_cashflow += discounted_cashflow
    return total_discounted_cashflow

df['MtM_leg1'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'fixed'), axis=1)
df['MtM_leg2'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'float'), axis=1)


# 1. Define a bumped discount rate curve function:
def bumped_discount_rate_curve(date):
    months_to_maturity = (date - datetime.now()).days // 30
    if months_to_maturity < 0:
        months_to_maturity = float_rates_df['MonthsToMaturity'].min()
    reindexed_df = float_rates_df.set_index('MonthsToMaturity').reindex(float_rates_df['MonthsToMaturity'], method='nearest')
    # Bump the rate by 0.0001
    bumped_rate = reindexed_df['Rate'].get(months_to_maturity, reindexed_df['Rate'].iloc[0]) + 0.01
    return bumped_rate / 100

# 2. Recalculate leg2 value using the bumped rate:
def calculate_bumped_discounted_cashflow(row, type):
    total_discounted_cashflow = 0
    months_to_maturity = (row['expirationDate'] - row['effectiveDate']).days // 30
    for date in row['cashflow_dates']:
        if type == 'float':
            float_rate = get_floating_rate(months_to_maturity) / 100
            cashflow = row['leg1NotionalAmount'] * float_rate
        else:
            cashflow = row['leg1NotionalAmount'] * row['leg1FixedRate']
        time_difference = (date - datetime.now()).days / 365.0
        discounted_cashflow = cashflow / (1 + bumped_discount_rate_curve(date))**time_difference
        total_discounted_cashflow += discounted_cashflow
    return total_discounted_cashflow

df['MtM_leg2_bumped'] = df.apply(lambda row: calculate_bumped_discounted_cashflow(row, 'float'), axis=1)

# 3. Calculate the risk:
df['total_delta'] = (df['MtM_leg2'] - df['MtM_leg2_bumped']).abs()


df.drop(columns=['cashflow_dates'], inplace=True)
df.drop(columns=['MtM_leg2_bumped'], inplace=True)


df.to_csv(your_path + '3.Cash_Risk/' + data_file + '_Cash_Risk.csv', index=False)





['USD-Federal Funds-H.15-OIS-COMPOUND' 'USD-SOFR-OIS Compound'
 'GBP-SONIA-OIS Compound' 'EUR-EURIBOR' 'EUR-EuroSTR-OIS Compound']
